In [1]:
import datetime
import itertools
import math
import os
from itertools import tee, repeat
import numpy as np
import pandas as pd
from progressbar import ProgressBar
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from math import sqrt
from sklearn.metrics import mean_squared_error, accuracy_score

In [2]:
os.chdir('../')
os.getcwd()

'C:\\Users\\Asus S\\sakce_git\\process\\Sprint4\\gucci_mane'

In [3]:
train_path = './data/road-train.csv'
test_path = './data/road-test.csv'

In [4]:
'''For accuracy and rmse - do it for all three new columns'''
def pre_data(train_path: str, test_path: str) -> object:

    if train_path == './data/2018-train.csv':
        df2_test = pd.read_csv(test_path, encoding = "ISO-8859-1",
        error_bad_lines = False, dtype = {'event org:resource': str}, engine='python')
        df2_train = pd.read_csv(train_path, encoding = "ISO-8859-1",
        error_bad_lines = False, dtype = {'event org:resource': str}, engine='python')
    else:
        df2_test = pd.read_csv(train_path)
        df2_train = pd.read_csv(test_path)

    # Convert from string to datetime
    df2_train['event time:timestamp'] = pd.to_datetime(df2_train['event time:timestamp'])
    df2_test['event time:timestamp'] = pd.to_datetime(df2_test['event time:timestamp'])

    # Concatenate the datasets
    df2 = pd.concat([df2_train, df2_test])

    # Sort the values and reset the index accordingly
    df2 = df2.sort_values(by=['event time:timestamp'])
    df2 = df2.reset_index(drop=True)

    # Create dataframe with first events of each case in chronological order
    df_cases_first_event = df2.drop_duplicates(subset='case concept:name', keep='first')
    df_cases_first_event = df_cases_first_event.reset_index(drop=True)

    # Split the cases in 80-20
    index80 = int(len(df_cases_first_event) * 80 / 100 - 1)
    df_cases_train = df_cases_first_event.loc[:index80]
    df_cases_test = df_cases_first_event.loc[index80 + 1:]

    # Training and test set complete
    df_train_new = df2.loc[df2['case concept:name'].isin(list(df_cases_train['case concept:name']))]
    df_test_new = df2.loc[df2['case concept:name'].isin(list(df_cases_test['case concept:name']))]

    # Setting the 'now' (as the last event of the last case in the training set)
    # now = df_train_new[df_train_new['case concept:name']==df_cases_train.iloc[-1]\
    #    ['case concept:name']].iloc[-1]['event time:timestamp']
    if 'case LoanGoal' in df2_train:
        now = pd.Timestamp('2016-12-08')
    elif len(df2_train.columns) == 5:
        now = pd.Timestamp('2012-12-12')
    elif 'case AMOUNT_REQ' in df2_train:
        now = pd.Timestamp('2012-10-02')
    else:
        now = pd.Timestamp('2017-11-12')

    # Training and test set before 'now'
    df_train_new_now = df_train_new.loc[(df_train_new['event time:timestamp'] <= now)]
    df_test_new_now = df_test_new.loc[(df_test_new['event time:timestamp'] <= now)]

    return df_train_new_now, df_test_new_now

In [5]:
data_train, data_test = pre_data(train_path, test_path)

## Decision Tree to output file for LR

To skip running the whole DT, run the cell that reads a different csv file(which is exactly what the decision tree would output. 

In [9]:
def de_tree(data_train, data_test):
    
    data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])
    data_train = data_train.sort_values(by=['case concept:name', 'event time:timestamp'])

    data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])
    data_test = data_test.sort_values(by=['case concept:name', 'event time:timestamp'])

    # LOAN DATA
    if 'case LoanGoal' in data_train.columns:
        data_train['case LoanGoal'] = data_train['case LoanGoal'].apply(
            lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)

        data_test['case LoanGoal'] = data_test['case LoanGoal'].apply(
            lambda x: 'Other - see explanation' if x == 'Other, see explanation' else x)

    pt1 = data_train.columns.get_loc('case concept:name') + 1
    pt2 = data_train.columns.get_loc('event concept:name') + 1
    pt3 = data_train.columns.get_loc('event time:timestamp') + 1

    # DANGER
    cases = list(data_train['event concept:name'].unique()) + list(data_test['event concept:name'].unique())
    cases.append('New Case')
    cases = list(set(cases))

    # 1. Train Data

    # DANGER
    data_train.to_csv("./build/fixed.csv")

    log = dict()
    with open('./build/fixed.csv', 'r') as file:
        next(file)
        for line in file:
            line = line.strip()
            if len(line) == 0:
                continue
            parts = line.split(',')
            caseid = parts[pt1]

            task = parts[pt2]
            # added two lines
            # first one to have numeric labels for weekdays, +len cases to prevent same labeling
            # last one is only for the predicted_df in the end
            timestamp = pd.to_datetime(parts[pt3]).weekday() + len(cases)
            timestamp_full = parts[pt3]
            if caseid not in log:
                log[caseid] = [[], [], []]

            log[caseid][0].append(task)

            # append the timestamps.
            log[caseid][1].append(timestamp_full)
            log[caseid][2].append(timestamp)

    file.close()

    os.remove('./build/fixed.csv')

    for i in log.keys():  # updating the dictionary to contain also all next events
        current = log[i][0]  # recording the cuurent case' events

        real_next = current[1:]  # next real events
        real_next.append('New Case')  # adding a 'new case' as real next event for every last event

        log[i].append(real_next)  # adding the real next events to the log file

    # 2. Test Data

    # DANGER
    data_test.to_csv("./build/fixed_test.csv")

    log_test = dict()
    with open('./build/fixed_test.csv', 'r') as file:
        next(file)
        for line in file:
            line = line.strip()
            if len(line) == 0:
                continue
            parts = line.split(',')
            caseid = parts[pt1]
            # same as training
            task = parts[pt2]
            timestamp = pd.to_datetime(parts[pt3]).weekday() + len(cases)
            timestamp_full = parts[pt3]

            if caseid not in log_test:
                log_test[caseid] = [[], [], []]

            log_test[caseid][0].append(task)
            log_test[caseid][1].append(timestamp_full)
            log_test[caseid][2].append(timestamp)

    file.close()

    os.remove('./build/fixed_test.csv')

    """Fixing a bug of cases that are in the test data but are incomplete due to the train-test split."""

    bugs = []

    for i in log_test.keys():  # recording the cases which have events cut because of the train - test split
        if len(log_test[i][0]) == 1:
            bugs.append(i)

    for x in bugs:  # deleting the above mentioned events
        del log_test[x]
        data_test.drop(data_test.index[data_test['case concept:name'] == x], inplace=True)

    for i in log_test.keys():
        current = log_test[i][0]  # current case' events

        real_next = current[1:]  # next real events
        real_next.append('New Case')  # adding a 'new case' as real next event for every last event
        log_test[i].append(real_next)

        # 3. Storing the data

    #  new dictionary that will contain for every position(key) the observed traces and next events for each trace(values)
    #  so case [A, B, C] would be saved as {0:[[A],[B]], 1: [[A,B], [C]], 2: [[A, B, C], [New Case]]}
    train_data = {}
    pbar = ProgressBar()
    print('[12/18]')

    for i in pbar(log.keys()):
        count = 0
        for x in log[i][0]:
            case = log[i][0]
            time = log[i][2]  # DANGER

            if count not in train_data:  # making the two lists in the dictionary
                train_data[count] = [[], [],
                                     []]  # list 1 is all for all traces of the position, list 2 is for all next events
            # DANGER EXTRA LIST
            train_data[count][0].append(case[:count + 1])  # appending the trace
            train_data[count][2].append(time[count])  # DANGER
            if count < len(case) - 1:
                train_data[count][1].append(case[count + 1])  # appending the next event of the trace

            elif count == len(case) - 1:
                train_data[count][1].append('New Case')

            count += 1

    #  repeating the same process on the test data
    test_data = {}
    pbar = ProgressBar()
    print('[13/18]')
    for i in pbar(log_test.keys()):
        count = 0
        for x in log_test[i][0]:
            case = log_test[i][0]
            time = log_test[i][2]  # DANGER

            if count not in test_data:
                test_data[count] = [[], [], []]

            test_data[count][0].append(case[:count + 1])  # appending the trace
            test_data[count][2].append(time[count])  # DANGER

            if count < len(case) - 1:
                test_data[count][1].append(case[count + 1])  # appending the next event of the trace

            elif count == len(case) - 1:
                test_data[count][1].append('New Case')

            count += 1

    # 4. Encoding

    # encoding all unique event names of all the data into integers

    le = preprocessing.LabelEncoder()
    le.fit(cases)  # encoding all event names into integers

    ### 4.1 TRAIN

    pbar = ProgressBar()
    print('[14/18]')

    for i in pbar(train_data.keys()):  # the dictionaries from above are encoded into integers

        encoded = []
        for trace in train_data[i][0]:  # encoding all strings of a trace, can be multiple if case lenght is more than 2
            local_encoded = []
            for event in trace:
                local_encoded.append(int(le.transform([event])))  # transforming into integer
            encoded.append(local_encoded)

        train_data[i][0] = np.array(encoded)  # making the list with integers into array so the tree can take it

        encoded_next = []  # encoding all strings of next events for a trace, its always length 1 !
        for g in train_data[i][1]:
            encoded_next.append(int(le.transform([g])))  # transforming into integer

        train_data[i][1] = np.array(encoded_next)  # making the list with integers into array

    ### 4.2 TEST

    # repeating the procedure from above on the test data

    pbar = ProgressBar()
    print('[15/18]')

    for i in pbar(test_data.keys()):

        encoded = []
        for trace in test_data[i][0]:
            local_encoded = []
            for event in trace:
                local_encoded.append(int(le.transform([event])))
            encoded.append(local_encoded)

        test_data[i][0] = np.array(encoded)

        encoded_next = []
        for g in test_data[i][1]:
            encoded_next.append(int(le.transform([g])))

        test_data[i][1] = np.array(encoded_next)

    # 5. Training the decision tree

    # Function for training decision tree for any given position (as long as the position is in the train data)

    # DANGER

    def decision_tree(pos):
        x_train = train_data[pos][0]
        # combine full trace data with weekday
        x_week = np.array(train_data[pos][2]).reshape(-1, 1)
        x_new = np.concatenate((x_train, x_week), axis=1)
        y_train = train_data[pos][1]

        classifier = DecisionTreeClassifier()
        classifier.fit(x_new, y_train)

        return classifier

    predictors = {}  # dictionary to contain all decision trees given the position
    #  key - position, value - decision tree for that position
    pbar = ProgressBar()
    print('[16/18]')

    for i in pbar(test_data.keys()):
        if i > len(train_data) - 1:
            predictors[i] = decision_tree(len(train_data) - 1)

        else:
            predictors[i] = decision_tree(i)

    # 6. Adding predictions

    pbar = ProgressBar()
    print('[17/18]')

    for i in pbar(
            log_test.keys()):  # adding an array with the encoding to the log_test dict. for every case in the test
        current = log_test[i][0]

        encoded = []  # list will contain all event names encoded into integers
        for g in current:
            encoded.append(int(le.transform([g])))
        encoded = np.array(encoded)
        log_test[i].append(encoded)

    def update_tree(case):

        case = case.tolist()

        count = 0
        for x in case:  # case is an array
            if count not in train_data:  # making the two lists in the dictionary
                # list 1 is all for all traces of the position, list 2 is for all next events
                train_data[count] = [np.array([]), np.array([])]
            train_data[count][0] = train_data[count][0].tolist()
            train_data[count][1] = train_data[count][1].tolist()

            train_data[count][0].append(case[:count + 1])  # appending the trace

            if count < len(case) - 1:
                train_data[count][1].append(case[count + 1])  # appending the next event of the trace

            elif count == len(case) - 1:
                train_data[count][1].append(int(le.transform(['New Case'])))

            train_data[count][0] = np.array(train_data[count][0])
            train_data[count][1] = np.array(train_data[count][1])

            predictors[count] = decision_tree(count)

            count += 1

    pbar = ProgressBar()
    print('[18/18]')

    for i in pbar(log_test.keys()):  # making predictions for every case in the log_test dict

        current_encoded = log_test[i][4]  # DANGER
        times = log_test[i][1]
        weeks = log_test[i][2]
        predictions = []  # list that will contain all predictions for a given case
        count = 0

        for x in current_encoded:

            # the if-else is a checks whether the case length is more than any case length observed in the train data
            if count >= len(train_data) - 1:  # if its in the train data we call the appropriate decision tree

                tree = predictors[len(train_data) - 1]
                p_trace = current_encoded[:(len(train_data))].reshape(-1, len(train_data))
                # Create new array with full trace and weekdays, same as with train
                p_weeks = np.array(weeks[len(train_data) - 1]).reshape(-1, 1)
                p_new = np.concatenate((p_trace, p_weeks), axis=1)
                pred = tree.predict(p_new)
                pred_string = le.inverse_transform(pred)[0]
                predictions.append(pred_string)



            else:  # if its not in the train data then we use the last observed decision tree from the train data

                tree = predictors[count]
                p_trace = current_encoded[:count + 1].reshape(-1, count + 1)
                # same as above
                p_weeks = np.array(weeks[count]).reshape(-1, 1)
                p_new = np.concatenate((p_trace, p_weeks), axis=1)
                pred = tree.predict(p_new)
                pred_string = le.inverse_transform(pred)[0]
                predictions.append(pred_string)

            count += 1

        log_test[i].append(predictions)  # adding all predictions to the log_test of the current case

        # UNCOMMENT THE LINE BELOW FOR ONLINE TRAINING

        # update_tree(current_encoded)

    # 7. Evaluation

    # making lists for every column we will have in the frame
    case_names = []
    event_names = []
    timestamp = []
    p_event = []
    current_real = []

    for i in log_test.keys():  # appending the right things to every list from the log_test file
        for x in range(len(log_test[i][0])):
            case_names.append(i)
            event_names.append(log_test[i][0][x])
            timestamp.append(log_test[i][1][x])
            p_event.append(log_test[i][-1][x])
            current_real.append(log_test[i][3][x])

    frame_dict = {'Case_ID': case_names, 'Event_Name': event_names,
                  'TimeStamp': timestamp, 'Next_Event': current_real, 'DTree_Event': p_event}
    
    predicted_df = pd.DataFrame.from_dict(frame_dict)
    
    return predicted_df, le, data_train, data_test

predicted_df, le, data_train, data_test = de_tree(data_train, data_test)

os.chdir('reg_test')

np.save('classes.npy', le.classes_)

data_train.to_csv('./data_train.csv', index = False)
data_test.to_csv('./data_test.csv', index = False)

predicted_df.to_csv('./road_tree_csv.csv', index = False)

[12/18]


100% |########################################################################|


[13/18]


100% |########################################################################|


[14/18]


...

100% |########################################################################|


[17/18]


100% |########################################################################|


[18/18]


100% |########################################################################|


In [7]:
# Run this, if previous is not run.
os.chdir('reg_test')

le = preprocessing.LabelEncoder()
le.classes_ = np.load('classes.npy')

data_train = pd.read_csv('./data_train.csv')
data_test = pd.read_csv('./data_test.csv')
predicted_df = pd.read_csv('./road_tree_csv.csv')

data_train['event time:timestamp'] = pd.to_datetime(data_train['event time:timestamp'])
data_test['event time:timestamp'] = pd.to_datetime(data_test['event time:timestamp'])
predicted_df['TimeStamp'] = pd.to_datetime(predicted_df['TimeStamp'])

### Linear Regression

In [ ]:
### We can wrap it in a function to run all at once, or keep outside to run cells individually

#def lin_reg(data_train, data_test, predicted_df):
#print("Timestamp Linear Regression Prediction...")

train = data_train.copy()
test = data_test.copy()

# Add new useful columns for the model train
train['position_event'] = train.groupby('case concept:name').cumcount()
train['position_event'] = train['position_event'] + 1
train['week_day'] = train['event time:timestamp'].dt.dayofweek

# Encoding all event names into integers
cases = train['event concept:name'].unique().tolist()
cases.insert(0, 'New Case')
le_case = preprocessing.LabelEncoder()
le_case.fit(cases)

# Encoding lifecycle into integers
life = train['event lifecycle:transition'].unique().tolist()
le_life = preprocessing.LabelEncoder()
le_life.fit(life)

# Preprocess data for model train
# Event poistion
x_train_position = np.array(train['position_event']).reshape(-1, 1)[:]
# Previous event
x_train_prev = list(train['event concept:name'])
x_train_prev = le_case.transform(x_train_prev)
x_train_prev = np.array(x_train_prev).reshape(-1, 1)[:]
# Event
x_train_event = list(train['event concept:name'])
x_train_event.insert(len(train), 'New Case')
x_train_event = le_case.transform(x_train_event)
x_train_event = np.array(x_train_event).reshape(-1, 1)[1:]
# Day of the week previous event event
x_train_week = list(train['week_day'])
x_train_week = np.array(x_train_week).reshape(-1, 1)[:]
# Timestamp event
train[['event time:timestamp']] = train[['event time:timestamp']].astype(str)
x_train_date = list(train['event time:timestamp'])
x_train_date.insert(len(train), None)
x_train_date = np.array(x_train_date).reshape(-1, 1)[1:]
# Timestamp previous event
x_train_date_prev = list(train['event time:timestamp'])
x_train_date_prev = np.array(x_train_date_prev).reshape(-1, 1)[:]
# Event Lifecycle
x_train_life = list(train['event lifecycle:transition'])
x_train_life = le_life.transform(x_train_life)
x_train_life = np.array(x_train_life).reshape(-1, 1)[:]


# Length case for train set
cases = train.groupby(['case concept:name'])
per_case = pd.DataFrame({'no of events': cases['eventID '].count()})
lst_per_case = per_case["no of events"].tolist()
case_length = []
for length in lst_per_case:
    case_length.extend(repeat(length, length))
x_train_length_case = np.array(case_length).reshape(-1, 1)[:]


# Combine features for the model train
x_train_new = np.concatenate((x_train_position, x_train_prev, x_train_event, x_train_week, x_train_date,
                              x_train_date_prev, x_train_length_case, x_train_life), axis=1)

# Add features to new dataframe train
df_train = pd.DataFrame(data=x_train_new,
                        columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev',
                                 'case_length', 'lifecycle'])
df_train.loc[df_train['position_event'] == df_train['case_length'], 'event'] = int(le.transform(['New Case']))
df_train[['date', 'date_prev']] = df_train[['date', 'date_prev']].apply(pd.to_datetime)
df_train.loc[df_train['event'] == int(le.transform(['New Case'])), 'date'] = None
df_train['in_between'] = (df_train['date'] - df_train['date_prev']).dt.total_seconds()
df_train.loc[df_train['event'] == int(le.transform(['New Case'])), 'in_between'] = 0
df_train[['case_length']] = df_train[['case_length']].astype(int)

# Train Dummies

# Implementing dummies train
df_train = pd.get_dummies(df_train, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_train = df_train.drop(['date', 'date_prev'], 1)

# Test Data Preprocessing

# Add new useful columns for the model test
test['position_event'] = test.groupby('case concept:name').cumcount()
test['position_event'] = test['position_event'] + 1
test['week_day'] = test['event time:timestamp'].dt.dayofweek
predicted_events = predicted_df['DTree_Event'][:].tolist()
test['pred_event'] = predicted_events

# Preprocess data for model test
# Event poistion
x_test_position = np.array(test['position_event']).reshape(-1, 1)[:]
# Previous event
x_test_prev = test['event concept:name'].tolist()
x_test_prev = le_case.transform(x_test_prev)
x_test_prev = np.array(x_test_prev).reshape(-1, 1)[:]
# Predicted Event
x_test_event = test['pred_event'].tolist()
x_test_event = le_case.transform(x_test_event)
x_test_event = np.array(x_test_event).reshape(-1, 1)[:]
# Day of the week previous event
x_test_week = test['week_day'].tolist()
x_test_week = np.array(x_test_week).reshape(-1, 1)[:]
# Timestamp event
test[['event time:timestamp']] = test[['event time:timestamp']].astype(str)
x_test_date = list(test['event time:timestamp'])
x_test_date.insert(len(test), None)
x_test_date = np.array(x_test_date).reshape(-1, 1)[1:]
# Timestamp previous event
x_test_date_prev = list(test['event time:timestamp'])
x_test_date_prev = np.array(x_test_date_prev).reshape(-1, 1)[:]
# Event Lifecycle
x_test_life = test['event lifecycle:transition'].tolist()
x_test_life = le_life.transform(x_test_life)
x_test_life = np.array(x_test_life).reshape(-1, 1)[:]


# Length case for test set
test_cases = test.groupby(['case concept:name'])
per_case_test = pd.DataFrame({'no of events': test_cases['eventID '].count()})
lst_per_case_test = per_case_test["no of events"].tolist()
case_length_test = []
for length in lst_per_case_test:
    case_length_test.extend(repeat(length, length))
x_test_length_case = np.array(case_length_test).reshape(-1, 1)[:]

# Combine features for the model test
x_test_new = np.concatenate((x_test_position, x_test_prev, x_test_event, x_test_week, x_test_date, x_test_date_prev,
                             x_test_length_case, x_test_life), axis=1)

# Add features to new dataframe test
df_test = pd.DataFrame(data=x_test_new,
                       columns=['position_event', 'prev_event', 'event', 'week_day_prev', 'date', 'date_prev',
                                'case_length', 'lifecycle'])
df_test.loc[df_test['position_event'] == df_test['case_length'], 'date'] = None
df_test[['date', 'date_prev']] = df_test[['date', 'date_prev']].apply(pd.to_datetime)
df_test['in_between'] = (df_test['date'] - df_test['date_prev']).dt.total_seconds()
df_test.loc[df_test['position_event'] == df_test['case_length'], 'in_between'] = 0
df_test[['case_length']] = df_test[['case_length']].astype(int)

# Remove cases with more events than the cases in the train set
df_test = df_test[df_test['case_length'] <= max(df_train['case_length'])]

# Test Dummies

# Implementing dummies test
df_test_d = pd.get_dummies(df_test, columns=['event', 'prev_event', 'week_day_prev', 'position_event', 'lifecycle'])
df_test_d = df_test_d.drop(['date', 'date_prev'], 1)

# Feature selection and model training

col_train = df_train.columns
col_test = df_test_d.columns
features = set(col_train).intersection(col_test)
features.discard('in_between')
X_train = df_train[features]  # Features
y_train = df_train['in_between']  # Target variable
X_test = df_test_d[features]  # Features
y_test = df_test_d['in_between']  # Target variable

# Training the algorithm
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Evaluation

# Workaround to get rid of negative values
# All 'New Case' to 0 and others to absolute value: RMSE 166.7941 (days)
y_pred = regressor.predict(X_test)
df_predictions = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
df_pred_fin = pd.concat([df_test, df_predictions], axis=1)
df_pred_fin.loc[df_pred_fin['event'] == int(le.transform(['New Case'])), 'Predicted'] = 0
y_pred = df_pred_fin['Predicted']
y_pred = abs(y_pred)
df_predictions = pd.DataFrame({'DTree_Actual_TimeDiff': y_test, 'DTree_TimeDiff': y_pred})

predicted_df = pd.concat([predicted_df, df_predictions], axis=1)

predicted_df = predicted_df[predicted_df['DTree_Actual_TimeDiff'].notna()]
predicted_df = predicted_df[predicted_df['DTree_TimeDiff'].notna()]

#return predicted_df

In [ ]:
def evaluate(d_tree):
    d_tree = predicted_df.copy()
    event_real = np.array(d_tree['Next_Event'])  # taking next event col. as an array
    event_pred = np.array(d_tree['DTree_Event'])  # taking the predictions as an array

    acc_tree = accuracy_score(event_real, event_pred)  # calculates the accuracy based on the both arrays
    print('Accuracy for event prediction DECISION TREE: {}%'.format(round(acc_tree, 2) * 100))

    # RMSE D Tree
    time_real_tree = np.array(d_tree['DTree_Actual_TimeDiff'])
    time_pred_tree = np.array(d_tree['DTree_TimeDiff'])

    rms_tree = sqrt(mean_squared_error(time_real_tree, time_pred_tree))/60/60/24

    return print('Root mean squared error for time difference prediction DECISION TREE: {}'.format(round(rms_tree, 2))) 

In [ ]:
evaluate(predicted_df)